<h1> 📘 Biofilter3R — Reports: Gene → SNP Annotation </h1>

This tutorial shows how to use the Reports API to generate standardized annotation outputs from Genes → SNPs using Biofilter.
The main report in this notebook resolves SNPs by genomic overlap between EntityLocations (genes, build 38) and VariantSNP (rsID positions, build 37/38).

### Methods

* report.list() — List all available reports.
* report.explain("<report>") — Print a detailed explanation of a report (inputs, outputs, rules).
* report.available_columns("<report>") — Show all optional output columns supported by the report.
* report.run("<report>", **params) — Execute a report and return a pandas DataFrame.
* report.run_example("<report>") — Run a report using its built-in example_input().

### Available Reports

* gene_to_snp — Given a list of genes (symbols/IDs/aliases), returns SNPs overlapping each gene region (Build 38 gene coordinates + SNP positions in Build 38 and/or Build 37).

--------

### Start Biofilter3R

In [1]:
from biofilter import Biofilter
bf = Biofilter()

[INFO] ════════════════════════════════════
[INFO] 🚀 Initializing Biofilter3R
[INFO]    • Version: 3.2.0
[INFO]    • Debug mode: False
[INFO]    • Config: /home/bioadmin/biofilter/.biofilter.toml
[INFO] ════════════════════════════════════
[INFO] 🔌 Database connection established
[INFO]    • Engine: postgresql+psycopg2
[INFO]    • Host:   localhost
[INFO]    • DB:     biofilter
[INFO]    • Time:   1.3 ms
[INFO] ════════════════════════════════════


In [3]:
# Before
import time
start_time = time.time()

df = bf.report.run_example("gene_to_snp")
end_time = time.time() - start_time
print(f"Δ: {end_time:.0f}ms")
df.head()

Δ: 290ms


,Input Gene,Matched Name,Alias Type,Alias Source,HGNC Symbol,Gene Entity ID,Gene Build,Gene Chr (23:X/24:Y),Gene Start (Build 38),Gene End (Build 38),...,Variant Type,Variant ID,SNP Chr (23:X/24:Y),SNP Pos (Build 38),SNP Pos (Build 37),Ref Allele,Alt Allele,DataSource,SourceSystem,Note
0,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,1451,19,58354491,58865857,G,A/C/T,dbsnp_chr19,NCBI,NaN
1,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893180,19,58353725,58865091,A,C/G/T,dbsnp_chr19,NCBI,NaN
2,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893181,19,58353716,58865082,A,C/G/T,dbsnp_chr19,NCBI,NaN
3,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893182,19,58353702,58865068,T,A/C,dbsnp_chr19,NCBI,NaN
4,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893183,19,58353688,58865054,T,A/C/G,dbsnp_chr19,NCBI,NaN


In [4]:
# After
import time
start_time = time.time()

df = bf.report.run_example("gene_to_snp")
end_time = time.time() - start_time
print(f"Δ: {end_time:.0f}s")
print(len(df))
df.head()

Δ: 0s
7961


,Input Gene,Matched Name,Alias Type,Alias Source,HGNC Symbol,Gene Entity ID,Gene Build,Gene Chr (23:X/24:Y),Gene Start (Build 38),Gene End (Build 38),...,Variant Type,Variant ID,SNP Chr (23:X/24:Y),SNP Pos (Build 38),SNP Pos (Build 37),Ref Allele,Alt Allele,DataSource,SourceSystem,Note
0,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,575574507,19,58344180,58855546,C,T,dbsnp_chr19,NCBI,NaN
1,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,375555639,19,58344182,58855548,T,C,dbsnp_chr19,NCBI,NaN
2,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,1418884653,19,58344187,58855553,G,A,dbsnp_chr19,NCBI,NaN
3,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,138605506,19,58344188,58855554,C,G,dbsnp_chr19,NCBI,NaN
4,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,2051900169,19,58344193,58855559,A,T,dbsnp_chr19,NCBI,NaN


------

### 1. Inspect the report

In [2]:
bf.report.explain("gene_to_snp")

🧬 GENE → SNP Report (v3.2.0)

This report takes gene identifiers (symbols, HGNC IDs, Entrez IDs, Ensembl IDs, synonyms)
and returns SNPs overlapping each gene region.

Rules:
- Gene regions come from EntityLocation and are currently available only for build 38.
- SNP lookup uses VariantSNP and overlaps are computed using build 38 coordinates (position_38).
- VariantSNP contains both position_38 and position_37; the report can display both.

Parameters:
- input_data: list[str] or path to .txt file (required)
- window_bp: int >= 0 (default: 1000). Extends the gene region +/- window_bp (build 38).
- assembly: "37" | "38" | None (default None).
    NOTE: assembly is OUTPUT-ONLY:
      - None: show both 'SNP Pos (Build 38)' and 'SNP Pos (Build 37)'
      - "38": show only 'SNP Pos (Build 38)'
      - "37": show only 'SNP Pos (Build 37)'
- output_columns: optional list[str] of DISPLAY NAMES (see available_columns()).

Usage:
    df = bf.report.run(
        "gene_to_snp",
        input_data=[

In [2]:
bf.report.available_columns("gene_to_snp")

['Input Gene', 'Matched Name', 'Alias Type', 'Alias Source', 'HGNC Symbol', 'Gene Entity ID', 'Gene Build', 'Gene Chr (23:X/24:Y)', 'Gene Start (Build 38)', 'Gene End (Build 38)', 'Gene Strand', 'Region Label', 'Variant Type', 'Variant ID', 'SNP Chr (23:X/24:Y)', 'SNP Pos (Build 38)', 'SNP Pos (Build 37)', 'Ref Allele', 'Alt Allele', 'DataSource', 'SourceSystem', 'Note']


---------

### 2. Run a simple example

In [4]:
df = bf.report.run_example("gene_to_snp")
df.head()

,Input Gene,Matched Name,Alias Type,Alias Source,HGNC Symbol,Gene Entity ID,Gene Build,Gene Chr (23:X/24:Y),Gene Start (Build 38),Gene End (Build 38),...,Variant Type,Variant ID,SNP Chr (23:X/24:Y),SNP Pos (Build 38),SNP Pos (Build 37),Ref Allele,Alt Allele,DataSource,SourceSystem,Note
0,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,1451,19,58354491,58865857,G,A/C/T,dbsnp_chr19,NCBI,NaN
1,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893180,19,58353725,58865091,A,C/G/T,dbsnp_chr19,NCBI,NaN
2,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893181,19,58353716,58865082,A,C/G/T,dbsnp_chr19,NCBI,NaN
3,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893182,19,58353702,58865068,T,A/C,dbsnp_chr19,NCBI,NaN
4,HGNC:5,HGNC:5,code,HGNC,A1BG,1,38,19,58345178,58353492,...,rs,893183,19,58353688,58865054,T,A/C/G,dbsnp_chr19,NCBI,NaN


In [4]:
bf.report.example_input("gene_to_snp")

['TXLNGY', 'HGNC:18473', '246126', 'ENSG00000131002', 'HGNC:5']


--------

### 3. Run with custom inputs

In [5]:
# Return Variants in Build 38
df = bf.report.run(
    "gene_to_snp",
    input_data = ['Tp53', 'Apoe'],
    assembly = 38
)
df.head()

,Input Gene,Matched Name,Alias Type,Alias Source,HGNC Symbol,Gene Entity ID,Gene Build,Gene Chr (23:X/24:Y),Gene Start (Build 38),Gene End (Build 38),...,Region Label,Variant Type,Variant ID,SNP Chr (23:X/24:Y),SNP Pos (Build 38),Ref Allele,Alt Allele,DataSource,SourceSystem,Note
0,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,1467532231,19,44902792,A,G,dbsnp_chr19,NCBI,<NA>
1,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,2513533110,19,44902793,A,G,dbsnp_chr19,NCBI,<NA>
2,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,2513533115,19,44902794,C,T,dbsnp_chr19,NCBI,<NA>
3,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,955657742,19,44902795,C,A/G/T,dbsnp_chr19,NCBI,<NA>
4,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,2513533119,19,44902797,A,G,dbsnp_chr19,NCBI,<NA>


In [6]:
# Return Variants in Build 37
df = bf.report.run(
    "gene_to_snp",
    input_data = ['Tp53', 'Apoe'],
    assembly = 37
)
df.head()

,Input Gene,Matched Name,Alias Type,Alias Source,HGNC Symbol,Gene Entity ID,Gene Build,Gene Chr (23:X/24:Y),Gene Start (Build 38),Gene End (Build 38),...,Region Label,Variant Type,Variant ID,SNP Chr (23:X/24:Y),SNP Pos (Build 37),Ref Allele,Alt Allele,DataSource,SourceSystem,Note
0,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,1467532231,19,45406049,A,G,dbsnp_chr19,NCBI,<NA>
1,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,2513533110,19,45406050,A,G,dbsnp_chr19,NCBI,<NA>
2,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,2513533115,19,45406051,C,T,dbsnp_chr19,NCBI,<NA>
3,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,955657742,19,45406052,C,A/G/T,dbsnp_chr19,NCBI,<NA>
4,Apoe,APOE,symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,rs,2513533119,19,45406054,A,G,dbsnp_chr19,NCBI,<NA>


In [2]:
# Return Variants in Build 37
df = bf.report.run("gene_to_snp", input_data = ['ad2'])
df.head()

,Input Gene,Matched Name,Alias Type,Alias Source,HGNC Symbol,Gene Entity ID,Gene Build,Gene Chr (23:X/24:Y),Gene Start (Build 38),Gene End (Build 38),...,Region Label,rsID,SNP Chr (23:X/24:Y),SNP Pos (Build 38),SNP Pos (Build 37),Ref Allele,Alt Allele,DataSource,SourceSystem,Note
0,ad2,AD2,prev_symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,7412,19,44908822,45412079,C,T,dbsnp_chr19,NCBI,None
1,ad2,AD2,prev_symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,7256173,19,44904398,45407655,C,T,dbsnp_chr19,NCBI,None
2,ad2,AD2,prev_symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,7259620,19,44904531,45407788,G,A/C,dbsnp_chr19,NCBI,None
3,ad2,AD2,prev_symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,9282609,19,44905856,45409113,C,T,dbsnp_chr19,NCBI,None
4,ad2,AD2,prev_symbol,HGNC,APOE,11434,38,19,44903787,44909396,...,None,10418135,19,44903541,45406798,G,C,dbsnp_chr19,NCBI,None


-------

### 4. Run with optional filters / parameters

In [8]:
df = bf.report.run(
    "gene_to_snp",
    input_data=["ENSG00000143801"],
    window_bp=0,        # optional (default: 1000)
    output_columns=[
        "HGNC Symbol",
        # "rsID",
        "SNP Chr (23:X/24:Y)",
        "SNP Pos (Build 38)",
        "Ref Allele",
        "Alt Allele", 
    ],  # optional
)
df.head()

,HGNC Symbol,SNP Chr (23:X/24:Y),SNP Pos (Build 38),Ref Allele,Alt Allele,Note
0,PSEN2,1,226870184,A,C,<NA>
1,PSEN2,1,226870192,G,T,<NA>
2,PSEN2,1,226870195,G,A,<NA>
3,PSEN2,1,226870202,T,A,<NA>
4,PSEN2,1,226870205,T,A,<NA>


In [3]:
# If you need to remember the columns names available here, just type:
bf.report.available_columns("gene_to_snp")